In [91]:
import torch

# Загрузка модели YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'custom', path='bag_belt_yolo.pt')
# Установка порога по уверенности и подавления
model.conf = 0.15  # Порог по уверенности
model.iou = 0.4  # Порог подавления



Using cache found in C:\Users\Дмитрий/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-21 Python-3.8.16 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080 Ti Laptop GPU, 16384MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20873139 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [92]:
def save_annotations_to_yolo_format(output_file, results, height_max, width_max):
    with open(output_file, 'w') as f:
        for result in results.xyxy[0]:
            label = result[-1]
            xmin, ymin, xmax, ymax = result[:4]

            # Получение координат центра объекта и его ширины и высоты
            width = xmax - xmin
            height = ymax - ymin
            x_center = xmin + width / 2
            y_center = ymin + height / 2

            # Запись аннотации в формате YOLO 1.1
            label = int(label)
            x_center_formatted = '{:.6f}'.format(x_center.item() / width_max)
            y_center_formatted = '{:.6f}'.format(y_center.item() / height_max)
            width_formatted = '{:.6f}'.format(width.item() / width_max)
            height_formatted = '{:.6f}'.format(height.item() / height_max)

            line = f"{label} {x_center_formatted} {y_center_formatted} {width_formatted} {height_formatted}\n"
            f.write(line)

In [72]:
'{:.6f}'.format(0.0597076)

'0.059708'

In [93]:
import os
import cv2
import numpy as np

image_folder = 'image_folder'
output_folder = 'results/obj_train_data'

# Создание папки для сохранения аннотаций, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Обработка изображений и сохранение аннотаций в формате YOLO 1.1
for image_file in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)

    height_max, width_max = image.shape[0], image.shape[1]

    # Инференс с использованием модели YOLOv5
    results = model(image)

    # Путь к файлу аннотации
    annotation_file = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}.txt")

    # Сохранение аннотаций в формате YOLO 1.1
    save_annotations_to_yolo_format(annotation_file, results, height_max, width_max)

print(f"Аннотации сохранены")

train_file = 'results/train.txt'  # Имя файла train.txt

# Создание пустого файла train.txt, если он не существует
if not os.path.exists(train_file):
    open(train_file, 'w').close()

# Запись имен файлов в train.txt
with open(train_file, 'w') as f:
    for image_file in os.listdir(image_folder):
        f.write('data/obj_train_data/' + image_file + '\n')


Аннотации сохранены


In [94]:
import zipfile

def create_zip_archive(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, arcname=os.path.relpath(file_path, folder_path))

# Указать путь к папке, файлы которой нужно добавить в zip-архив
folder_path = 'results'
# Указать путь для сохранения zip-архива
output_path = 'annotations.zip'

create_zip_archive(folder_path, output_path)

In [11]:
def convert_results_to_coco_format(results, image_file):
    coco_annotations = []
    #image_id = int(image_file.split('.')[0])

    for result in results.xyxy[0]:
        label = result[-1]
        xmin, ymin, xmax, ymax = result[:4]
        width, height = xmax - xmin, ymax - ymin
        print(xmin, ymin, xmax, ymax)

        coco_annotation = {
            "id": len(coco_annotations) + 1,
            "file_name": image_file,
            "category_id": int(label),
            "bbox": [xmin, ymin, width, height],
            "area": width * height,
            "iscrowd": 0
        }

        coco_annotations.append(coco_annotation)

    return coco_annotations

In [13]:
import os
import cv2
import numpy as np
import json

image_folder = 'image_folder'



output_file = 'example.json'

# Проверка наличия файла
if not os.path.exists(output_file):
    # Создание пустого JSON-файла
    with open(output_file, 'w') as f:
        json.dump({}, f)
    print(f"Создан пустой JSON-файл: {output_file}")
else:
    print(f"Файл уже существует: {foutput_file}")


annotations = []

# Получение списка изображений из папки
image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

# Выполнение инференса и сохранение аннотаций в формате COCO 1.0
for image_file in image_files[:2]:
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)

    # Инференс с использованием модели YOLOv5
    results = model(image)
    print(results)

    # Получение аннотаций из результатов инференса и добавление в список
    coco_annotations = convert_results_to_coco_format(results, image_file)
    annotations.extend(coco_annotations)

# Сохранение аннотаций в формате COCO JSON
coco_data = {
    "images": [],
    "annotations": annotations,
    "categories": []
}

with open(output_file, 'w') as f:
    json.dump(coco_data, f)

print(f"Аннотации сохранены в файле: {output_file}")


Создан пустой JSON-файл: example.json
image 1/1: 1080x1920 1 person, 1 head
Speed: 3.1ms pre-process, 130.8ms inference, 4.0ms NMS per image at shape (1, 3, 384, 640)
tensor(130.81345, device='cuda:0') tensor(648.97351, device='cuda:0') tensor(394.06155, device='cuda:0') tensor(1074.30127, device='cuda:0')
tensor(936.75684, device='cuda:0') tensor(590.53967, device='cuda:0') tensor(1062.51355, device='cuda:0') tensor(795.39026, device='cuda:0')
image 1/1: 1080x1920 1 person, 3 heads
Speed: 1.9ms pre-process, 100.2ms inference, 2.4ms NMS per image at shape (1, 3, 384, 640)
tensor(355.83731, device='cuda:0') tensor(592.87115, device='cuda:0') tensor(479.29358, device='cuda:0') tensor(747.11957, device='cuda:0')
tensor(727.71594, device='cuda:0') tensor(428.71152, device='cuda:0') tensor(874.00415, device='cuda:0') tensor(777.43518, device='cuda:0')
tensor(755.98010, device='cuda:0') tensor(432.72662, device='cuda:0') tensor(826.35632, device='cuda:0') tensor(524.10175, device='cuda:0')
t

TypeError: Object of type Tensor is not JSON serializable